In [ ]:
import tarfile
import pandas as pd
import os


def extract_tar(tar_filename, extract_path):
    with tarfile.open(tar_filename, 'r:gz') as tar:
        tar.extractall(path=extract_path)
        print(f"Извлечены файлы в {extract_path}")

def load_filters_from_csv(csv_filename):
    return pd.read_csv(csv_filename, header=None).values


tar_filename = 'best_pictures.tar.gz'
extract_path = 'best_pictures'  

extract_tar(tar_filename, extract_path)


extracted_files = os.listdir(extract_path)
print("Извлеченные файлы:", extracted_files)

algos_csv_path = os.path.join(extract_path, 'algos.csv')

filters = load_filters_from_csv(algos_csv_path)
print("Загруженные фильтры:\n", filters)

In [ ]:
import numpy as np
import pandas as pd
from scipy.ndimage import convolve
from sklearn.metrics import mean_squared_error
from itertools import permutations
import matplotlib.pyplot as plt  


filter1 = np.array([[-1.0, -0.5, 0.0],
                    [-0.5, 0.5, 0.5],
                    [0.0, 0.5, 1.0]])

filter2 = np.array([[0.0625, 0.0625, 0.0625],
                    [0.0625, 0.0625, 0.0625],
                    [0.0625, 0.0625, 0.0625]])


unknown_filter = np.array([[ 0.73119182, -0.03341565,  0.0136693 ],
 [ 0.71254906,  0.48218111, -0.0778814 ],
 [ 0.67657722,  0.31003467,  0.4760988 ]] )


def find_files(directory):

    images = {}
    outputs = {}

    for filename in os.listdir(directory):
        if filename.endswith('.png'):

            number = filename.split('.')[0]
            images[number] = os.path.join(directory, filename)
        elif filename.endswith('.txt'):
     
            number = filename.split('.')[0]
            outputs[number] = os.path.join(directory, filename)


    image_filenames = []
    output_filenames = []


    for number in sorted(images.keys()):
        image_filenames.append(images[number])
        if number in outputs:
            output_filenames.append(outputs[number])

    return image_filenames, output_filenames

def load_image_as_2d(image_path):
    """Загружает изображение как 2D массив (даже если оно цветное)"""
    img = plt.imread(image_path)
    if img.ndim == 3:
        img = img.mean(axis=2) 
    return img

def load_data(image_filenames, output_filenames):
    images = []
    expected_outputs = []

    for img_file, expected_file in zip(image_filenames, output_filenames):

        img = load_image_as_2d(img_file)
        images.append(img)

        expected_output = np.loadtxt(expected_file)
        expected_outputs.append(expected_output)

    return images, expected_outputs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


image_filenames, output_filenames = find_files('best_pictures')
images, expected_outputs = load_data(image_filenames, output_filenames)

images_tensor = torch.tensor(images,dtype=torch.float32)*255  # [N, H, W]
targets_tensor = torch.tensor(expected_outputs,dtype=torch.float32) # [N, H_out, W_out]


min_val = torch.min(images_tensor)
max_val = torch.max(images_tensor)

# Нормировка
#images_tensor = (images_tensor - min_val) / (max_val - min_val)*255

images_tensor = images_tensor.unsqueeze(1)  # [N, 1, H, W]
targets_tensor = targets_tensor.unsqueeze(1)  # [N, 1, H_out, W_out]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

class FullConvModel(nn.Module):
    def __init__(self, fixed_filter1, fixed_filter2):
        super().__init__()
        # (2->3->1)
        self.conv2 = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
        self.conv1 = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)


        with torch.no_grad():
            self.conv1.weight.data = torch.tensor(fixed_filter1, dtype=torch.float32).view(1, 1, 3, 3)
            self.conv2.weight.data = torch.tensor(fixed_filter2, dtype=torch.float32).view(1, 1, 3, 3)
            self.conv1.weight.requires_grad = False
            self.conv2.weight.requires_grad = False


        self.conv3 = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=True)
        nn.init.normal_(self.conv3.weight, mean=0, std=0.01)
        nn.init.zeros_(self.conv3.bias)  

    def forward(self, x):

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv1(x)
        return x

def train_full_model(model, dataloader, epochs=100, lr=0.001):
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    loss_history = []

    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_idx, (batch_images, batch_targets) in enumerate(dataloader):
            optimizer.zero_grad()
            outputs = model(batch_images)
            loss = criterion(outputs, batch_targets)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            epoch_loss += loss.item()

            #if batch_idx % 10 == 0:
                #print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.6f}")

        avg_loss = epoch_loss / len(dataloader)
        loss_history.append(avg_loss)
        print(f"Epoch {epoch} завершена. Avg Loss: {avg_loss:.6f}")

    return loss_history


dataset = TensorDataset(images_tensor, targets_tensor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


model = FullConvModel(filter1, filter2)
loss_history = train_full_model(model, dataloader, epochs=1000, lr=0.0001)

plt.plot(loss_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Full Conv Training (Batch)')
plt.show()


print("Обученный фильтр 3:\n", model.conv3.weight.data.squeeze().cpu().numpy())

In [ ]:

filters = [
    model.conv2.weight.data.squeeze().numpy(),  
    model.conv3.weight.data.squeeze().numpy(),  
    model.conv1.weight.data.squeeze().numpy() 
]


np.savetxt('reconstructed_algos.csv',
           np.vstack([f.flatten() for f in filters]),
           delimiter=',', fmt='%.6f')

In [ ]:
filters

Дальше можно присмотреться и увидеть, что значения свертки схожи с одной известной нам аналитически сверткой.

Методом пристального взора можно понять, что "верные" значения это округление этой свертки до 3 знака после запятой

Т.е.:

In [ ]:
[[0.12408582, 0.25154385, 0.12425872],
[0.2515448 , 0.49695498, 0.251543  ],
[0.12426438, 0.25153747, 0.12409208]]

-> -> ->

In [ ]:
[[0.125, 0.25, 0.125],
[0.25 , 0.5, 0.25  ],
[0.125, 0.25, 0.125]]

Вот и вся задача. Самое непонятное в ней это умножение/деление на 256 из-за нормировки входных данных

НИЖЕ КОД НЕ ПЕРЕПРОВЕРЯЛСЯ НА КОМПИЛЯЦИЮ!

Для поиска верной последовательности нужно было посмотреть как быстро падает Loss при обучении моделей с разными последовательностями слоёв. Здесь это и происходило в коде

In [ ]:
class FlexibleConvModel(nn.Module):
    def __init__(self, filters, order):
        super().__init__()
        self.order = order
        self.convs = nn.ModuleList([
            nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
            for _ in range(3)
        ])

       
        with torch.no_grad():
            for i, f in enumerate(filters[:2]):
                self.convs[i].weight.data = torch.tensor(f, dtype=torch.float32).view(1, 1, 3, 3)
                self.convs[i].weight.requires_grad = False  


        nn.init.normal_(self.convs[2].weight, mean=0, std=0.01)

    def forward(self, x):
        for i in self.order:
            x = self.convs[i](x)
        return x

In [ ]:
def train_with_order(filters, order, epochs=500, lr=0.0875):
    model = FlexibleConvModel(filters, order)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(images_tensor)
        loss = criterion(outputs, targets_tensor)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return losses

In [ ]:
import itertools
import matplotlib.pyplot as plt


all_orders = list(itertools.permutations([0, 1, 2]))


order_results = {}


for order in all_orders:
    print(f"Обучение для порядка: {order}")
    losses = train_with_order([filter1, filter2, None], order)
    order_results[order] = losses

In [ ]:
plt.figure(figsize=(12, 6))
for order, losses in order_results.items():
    plt.plot(losses, label=f'Порядок: {order}')

plt.xlabel('Эпоха')
plt.ylabel('Loss (MSE)')
plt.title('Сравнение сходимости для разных порядков фильтров')
plt.yscale('log')  
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
for i in order_results:
  print(order_results[i][-1])